In [538]:
'''
Created on 2016. 4. 26

@author: dato
@desc: cnlp hw8 (max entropy classifier)
'''
#-*- coding: utf-8 -*-

import io
from konlpy.tag import Kkma

In [2]:
data_file = './wsd_dataset.tsv'

In [28]:
f = io.open(data_file, 'r', encoding='utf-8')
lines = f.readlines()

total_doc = len(lines)
parsed_lines = [line.split('\t') for line in lines]

print 'total # of doc : ' + str(total_doc)

total # of doc : 4937


# split data set

In [243]:
apple = []
pear = []
potato = []

for parsed_line in parsed_lines:
    tmp = parsed_line[0].split('_')[0]
    
    if( tmp == unicode('사과'.decode('utf-8'))):
        apple.append(parsed_line)
    elif( tmp == unicode('배'.decode('utf-8'))):
        pear.append(parsed_line)
    else:
        potato.append(parsed_line)

In [466]:
print len(apple)
print len(pear)
print len(potato)

742
3868
327


# util func

In [250]:
# random sampling data from list


from random import randrange

def random_sample(list_data, portion):
    
    list_train = []
    list_test = []
    a = random.sample(xrange(len(list_data)), int(len(list_data)*portion))

    for x in range(len(list_data)):
        if x not in a:
            list_train.append(list_data[x])
        else:
            list_test.append(list_data[x])
    
    return list_train, list_test

# training function


In [550]:
from konlpy.tag import Kkma
kkma = Kkma()

def preprocess_train_data(list_data):
    
    fianl_list = []
    
    # 0: label     # 1: left_sentence     # 2 : label     # 3 : righ_sentence
    for data in list_data:
        feature = {}
        tokens=kkma.morphs(data[1])
        # prepare feature
        for token in tokens:
            if token in feature:
                feature[token] += 1
            else:
                feature[token] =1
                
        tokens=kkma.morphs(data[3])
        # prepare feature
        for token in tokens:
            if token in feature:
                feature[token] += 1
            else:
                feature[token] =1
                
        # prepare turple
        tmp = (feature, data[0])
        
        # add turple to list
        fianl_list.append(tmp)        

    return fianl_list

In [551]:
def preprocess_test_data(list_data):
    fianl_list = []
    final_label = []
    
    for data in list_data:
        feature = {}
        tokens=kkma.morphs(data[1])
        # prepare feature
        for token in tokens:
            if token in feature:
                feature[token] += 1
            else:
                feature[token] =1
                
        tokens=kkma.morphs(data[3])
        # prepare feature
        for token in tokens:
            if token in feature:
                feature[token] += 1
            else:
                feature[token] =1
                
        # add data 
        fianl_list.append(feature)
        
        # add label
        final_label.append(data[0])
        
    return fianl_list, final_label

In [552]:
def accuracy(label, result):
    result_dic = {}
      
    for x in range(len(label)):
   
        if label[x] == result[x]:
            if label[x] + 'T' in result_dic:
                result_dic[label[x]+'T'] += 1
            else:
                result_dic[label[x]+'T'] = 1
        else:
            if label[x] + 'F'+result[x] in result_dic:
                result_dic[label[x]+'F'+result[x]] += 1
            else:
                result_dic[label[x]+'F'+result[x]] = 1
            
    return result_dic

# dataset process

In [553]:
# divide dataset into 2 set
apple_train, apple_test = random_sample(apple, 0.2)
pear_train, pear_test = random_sample(pear, 0.2)
potato_train, potato_test = random_sample(potato, 0.2)

print 'apple train / test'
print len(apple_train)
print len(apple_test)
print 'pear train / test'
print len(pear_train)
print len(pear_test)
print 'potato train / test'
print len(potato_train)
print len(potato_test)

apple train / test
594
148
pear train / test
3095
773
potato train / test
262
65


In [470]:
apple_train_final = preprocess_train_data(apple_train)
apple_test_final, apple_test_label = preprocess_test_data(apple_test)

pear_train_final = preprocess_train_data(pear_train)
pear_test_final, pear_test_label = preprocess_test_data(pear_test)

potato_train_final = preprocess_train_data(potato_train)
potato_test_final, potato_test_label = preprocess_test_data(potato_test)

# MaxEnt classifier

In [531]:
from nltk.classify import maxent

encoding1 = maxent.TypedMaxentFeatureEncoding.train(apple_train_final, count_cutoff=3, alwayson_features=True)
me_classifier1 = maxent.MaxentClassifier.train(apple_train_final, bernoulli=False, encoding=encoding1, trace=0)
me_result1 = me_classifier1.classify_many(apple_test_final)

In [532]:
encoding2 = maxent.TypedMaxentFeatureEncoding.train(pear_train_final, count_cutoff=3, alwayson_features=True)
me_classifier2 = maxent.MaxentClassifier.train(pear_train_final, bernoulli=False, encoding=encoding2, trace=0)
me_result2 = me_classifier2.classify_many(pear_test_final)

In [533]:
encoding3 = maxent.TypedMaxentFeatureEncoding.train(potato_train_final, count_cutoff=3, alwayson_features=True)
me_classifier3 = maxent.MaxentClassifier.train(potato_train_final, bernoulli=False, encoding=encoding3, trace=0)
me_result3 = me_classifier3.classify_many(potato_test_final)

In [540]:
me_rdic1 = accuracy(apple_test_label, me_result1)
me_rdic2 = accuracy(pear_test_label, me_result2)
me_rdic3 = accuracy(potato_test_label, me_result3)

In [541]:
me_rdic1

{u'\uc0ac\uacfc__05/NNGF\uc0ac\uacfc__08/NNG': 8,
 u'\uc0ac\uacfc__05/NNGT': 65,
 u'\uc0ac\uacfc__08/NNGF\uc0ac\uacfc__05/NNG': 8,
 u'\uc0ac\uacfc__08/NNGT': 67}

In [542]:
me_rdic2

{u'\ubc30__01/NNGF\ubc30__02/NNG': 28,
 u'\ubc30__01/NNGF\ubc30__03/NNG': 1,
 u'\ubc30__01/NNGF\ubc30__09/NNG': 9,
 u'\ubc30__01/NNGT': 221,
 u'\ubc30__02/NNGF\ubc30__01/NNG': 26,
 u'\ubc30__02/NNGF\ubc30__03/NNG': 2,
 u'\ubc30__02/NNGF\ubc30__09/NNG': 11,
 u'\ubc30__02/NNGT': 189,
 u'\ubc30__03/NNGF\ubc30__01/NNG': 9,
 u'\ubc30__03/NNGF\ubc30__02/NNG': 11,
 u'\ubc30__03/NNGF\ubc30__09/NNG': 1,
 u'\ubc30__03/NNGT': 19,
 u'\ubc30__09/NNGF\ubc30__01/NNG': 9,
 u'\ubc30__09/NNGF\ubc30__02/NNG': 11,
 u'\ubc30__09/NNGF\ubc30__03/NNG': 1,
 u'\ubc30__09/NNGT': 225}

In [543]:
me_rdic3

{u'\uac10\uc790__01/NNGF\uac10\uc790__05/NNG': 1,
 u'\uac10\uc790__01/NNGT': 58,
 u'\uac10\uc790__05/NNGF\uac10\uc790__01/NNG': 3,
 u'\uac10\uc790__05/NNGT': 3}

# Naive Bayes Classifier

In [554]:
import nltk

nb_classifier1 = nltk.classify.NaiveBayesClassifier.train(apple_train_final)
nb_result1 = nb_classifier1.classify_many(apple_test_final)

nb_classifier2 = nltk.classify.NaiveBayesClassifier.train(pear_train_final)
nb_result2 = nb_classifier2.classify_many(pear_test_final)

nb_classifier3 = nltk.classify.NaiveBayesClassifier.train(potato_train_final)
nb_result3 = nb_classifier3.classify_many(potato_test_final)

In [555]:
nb_rdic1 = accuracy(apple_test_label, nb_result1)
nb_rdic2 = accuracy(pear_test_label, nb_result2)
nb_rdic3 = accuracy(potato_test_label, nb_result3)

In [556]:
nb_rdic1

{u'\uc0ac\uacfc__05/NNGF\uc0ac\uacfc__08/NNG': 6,
 u'\uc0ac\uacfc__05/NNGT': 67,
 u'\uc0ac\uacfc__08/NNGF\uc0ac\uacfc__05/NNG': 4,
 u'\uc0ac\uacfc__08/NNGT': 71}

In [557]:
nb_rdic2

{u'\ubc30__01/NNGF\ubc30__02/NNG': 11,
 u'\ubc30__01/NNGF\ubc30__03/NNG': 68,
 u'\ubc30__01/NNGF\ubc30__09/NNG': 8,
 u'\ubc30__01/NNGT': 172,
 u'\ubc30__02/NNGF\ubc30__01/NNG': 13,
 u'\ubc30__02/NNGF\ubc30__03/NNG': 59,
 u'\ubc30__02/NNGF\ubc30__09/NNG': 13,
 u'\ubc30__02/NNGT': 143,
 u'\ubc30__03/NNGF\ubc30__02/NNG': 2,
 u'\ubc30__03/NNGF\ubc30__09/NNG': 2,
 u'\ubc30__03/NNGT': 36,
 u'\ubc30__09/NNGF\ubc30__01/NNG': 3,
 u'\ubc30__09/NNGF\ubc30__02/NNG': 5,
 u'\ubc30__09/NNGF\ubc30__03/NNG': 35,
 u'\ubc30__09/NNGT': 203}

In [558]:
nb_rdic3

{u'\uac10\uc790__01/NNGF\uac10\uc790__05/NNG': 25,
 u'\uac10\uc790__01/NNGT': 34,
 u'\uac10\uc790__05/NNGT': 6}